In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from math import sqrt
import joblib

# Load your dataset
df = pd.read_csv('/content/drive/MyDrive/data/user_rating_clean.csv')

# Create a matrix of user-item ratings
user_item_matrix = df.pivot_table(index='User_Id', columns='Place_Id', values='Place_Ratings', fill_value=0)

# Normalize the ratings for collaborative filtering
user_item_matrix_normalized = user_item_matrix.sub(user_item_matrix.mean(axis=1), axis=0)

# Split the dataset into training and testing sets
train_data, test_data = train_test_split(df, test_size=0.2)

# Create user-item matrices for training and testing sets
train_user_item_matrix = train_data.pivot_table(index='User_Id', columns='Place_Id', values='Place_Ratings', fill_value=0)
test_user_item_matrix = test_data.pivot_table(index='User_Id', columns='Place_Id', values='Place_Ratings', fill_value=0)

# Train-test split for neural network
train_nn, test_nn = train_test_split(user_item_matrix_normalized.values, test_size=0.2)

# Function to predict ratings using cosine similarity
def predict_ratings(user_item_matrix, similarity_matrix):
    # Calculate the dot product of the similarity matrix and the user-item matrix
    pred = similarity_matrix.dot(user_item_matrix)
    pred /= np.array([np.abs(similarity_matrix).sum(axis=1)]).T
    return pred

# Function to calculate RMSE
def rmse(prediction, actual):
    prediction = prediction[actual.nonzero()].flatten()
    actual = actual[actual.nonzero()].flatten()
    return sqrt(mean_squared_error(prediction, actual))

# Build a more complex Neural Network for model
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(units=128, activation=tf.nn.relu, input_dim=train_nn.shape[1]))
model.add(tf.keras.layers.Dense(units=256, activation=tf.nn.relu))
model.add(tf.keras.layers.Dense(units=128, activation=tf.nn.relu))
model.add(tf.keras.layers.Dense(units=train_nn.shape[1], activation='linear'))
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss='mean_squared_error')

# Train the model for 100 epochs
num_epochs = 150
for epoch in range(num_epochs):

    model.fit(train_nn, train_nn, epochs=1, validation_data=(test_nn, test_nn), verbose=0)
    # Calculate user similarity using cosine similarity
    user_similarity = cosine_similarity(train_user_item_matrix)

    # Make rating predictions for travel destinations
    user_item_prediction = predict_ratings(train_user_item_matrix.values, user_similarity)

    # Calculate and print RMSE on the training data
    train_rmse = rmse(user_item_prediction, train_user_item_matrix.values)
    print(f'Epoch {epoch + 1}/{num_epochs} - Train RMSE: {train_rmse}')

# Get the predicted ratings for all users
all_users_ratings = pd.DataFrame(user_item_prediction, index=train_user_item_matrix.index, columns=train_user_item_matrix.columns)


Epoch 1/150 - Train RMSE: 2.8449737441942067
Epoch 2/150 - Train RMSE: 2.8449737441942067
Epoch 3/150 - Train RMSE: 2.8449737441942067
Epoch 4/150 - Train RMSE: 2.8449737441942067
Epoch 5/150 - Train RMSE: 2.8449737441942067
Epoch 6/150 - Train RMSE: 2.8449737441942067
Epoch 7/150 - Train RMSE: 2.8449737441942067
Epoch 8/150 - Train RMSE: 2.8449737441942067
Epoch 9/150 - Train RMSE: 2.8449737441942067
Epoch 10/150 - Train RMSE: 2.8449737441942067
Epoch 11/150 - Train RMSE: 2.8449737441942067
Epoch 12/150 - Train RMSE: 2.8449737441942067
Epoch 13/150 - Train RMSE: 2.8449737441942067
Epoch 14/150 - Train RMSE: 2.8449737441942067
Epoch 15/150 - Train RMSE: 2.8449737441942067
Epoch 16/150 - Train RMSE: 2.8449737441942067
Epoch 17/150 - Train RMSE: 2.8449737441942067
Epoch 18/150 - Train RMSE: 2.8449737441942067
Epoch 19/150 - Train RMSE: 2.8449737441942067
Epoch 20/150 - Train RMSE: 2.8449737441942067
Epoch 21/150 - Train RMSE: 2.8449737441942067
Epoch 22/150 - Train RMSE: 2.84497374419420

In [ ]:
# Choose a user for recommendations
user_id_for_recommendation = int(input("Masukkan user ID: "))

# Ensure that the entered user ID is valid
while user_id_for_recommendation not in train_user_item_matrix.index:
    print("User ID tidak ditemukan")
    user_id_for_recommendation = int(input("Masukkan user ID: "))

# Get the user's ratings
user_ratings = all_users_ratings.loc[user_id_for_recommendation]

# Get the top 10 recommendations for the user
top_n = 10
top_recommendations = user_ratings.sort_values(ascending=False).head(top_n)

# Display the top recommendations with 'Place_Name', 'Category', and 'City'
recommended_places_info = df[df['Place_Id'].isin(top_recommendations.index)][['Place_Id', 'Place_Name', 'Category', 'City']].drop_duplicates()
recommended_places_info = recommended_places_info.set_index('Place_Id')
recommended_places_info = recommended_places_info.loc[top_recommendations.index]

print(f'\nTop {top_n} Recommendations for User {user_id_for_recommendation}:\n')
print(recommended_places_info)

Masukkan user ID: 23

Top 10 Recommendations for User 23:

                               Place_Name       Category         City
Place_Id                                                             
142               Danau Summarecon Bekasi     Cagar Alam       Bekasi
177                   Inkarla Agro Wisata         Budaya      Cianjur
348              Pantai Timur Pangandaran     Cagar Alam  Pangandaran
18                             Bukit Moko     Cagar Alam      Bandung
156       Taman Wisata Matahari Waterpark  Taman Hiburan        Bogor
97                Kyotoku Floating Market         Budaya      Bandung
281                   Taman Kota Kuningan  Taman Hiburan     Kuningan
398               Gunung Tangkuban Parahu     Cagar Alam       Subang
165              Wisata Alam Ciung Wanara     Cagar Alam       Ciamis
260          Goa Dayeuh, Karawang Selatan     Cagar Alam     Karawang


In [ ]:
# Save the model to an h5 file
model.save("/content/drive/MyDrive/data/fitur1-1.h5")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
